In [2]:
import xarray as xr
import itertools

In [3]:
# Créer un dataset par chunk dask
def split_by_chunks(dataset):
    chunk_slices = {}
    for dim, chunks in dataset.chunks.items():
        slices = []
        start = 0
        for chunk in chunks:
            if start >= dataset.sizes[dim]:
                break
            stop = start + chunk
            slices.append(slice(start, stop))
            start = stop
        chunk_slices[dim] = slices
    for slices in itertools.product(*chunk_slices.values()):
        selection = dict(zip(chunk_slices.keys(), slices))
        yield dataset[selection]

In [7]:
input_path = ''
ds = xr.open_dataset(input_path, chunks={'latitude':78, 'longitude':99}) # taille des chunks à adapter en fonction du fichier
#ds = ds.drop_vars(['mlotst', 'so', 'zos'])
ds

<xarray.Dataset>
Dimensions:    (depth: 2, latitude: 191, longitude: 509, time: 9496)
Coordinates:
  * depth      (depth) float32 0.494025 40.34405
  * longitude  (longitude) float32 -6.0 -5.9166665 ... 36.25 36.333332
  * latitude   (latitude) float32 30.166666 30.25 30.333334 ... 45.916668 46.0
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 2018-12-31T12:00:00
Data variables:
    thetao     (time, depth, latitude, longitude) float32 dask.array<chunksize=(9496, 1, 78, 99), meta=np.ndarray>
Attributes:
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2020/08/10 10:27:25 MERCATOR OCEAN Ne...
    source:                             MERCATOR GLORYS12V1
    institution:                        MERCATOR OCEAN
    references:                         http://www.mercator-ocean.fr
    comment:                            CMEMS product
    Conventions:                        CF-1.4
    domain_name:                        GL12
    FROM_ORIGINAL_FILE__field_type:     mean
    field_date:                         2019-06-25 00:00:00
    field_julian_date:                  25377.0
    julian_day_unit:                    days since 1950-01-01 00:00:00
    forecast_range:                     
    forecast_type:                      
    bulletin_date:                      2019-06-26 00:00:00
    bulletin_type:                      operational
    FROM_ORIGINAL_FILE__longitude_min:  -180.0
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

In [5]:
chunked_ds = list(split_by_chunks(ds))

In [6]:
output_path = ''
p = [output_path+'Cut_'+str(i)+'.nc' for i in range(1,len(chunked_ds))]

In [ ]:
xr.save_mfdataset(datasets=chunked_ds, paths=p)